<a href="https://colab.research.google.com/github/markpedraza/Starter-Neural-Network/blob/main/FirstNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starter Neural Network project.
## - We will use the iris dataset to predict the class of a flower.
## - 4 features of the flower will be used
# Credits
## - This project was made by following a tutorial from "Codemy.com" on YouTube.
## - https://www.youtube.com/playlist?list=PLCC34OHNcOtpcgR9LEYSdi9r7XIbpkpK1


In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [74]:
# Create a Model class that inherits nn.Module
class Model(nn.Module):
  # Input layer (4 features of the flower) -->
  # H2 (n)
  # output (3 classes of iris flowers)
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc1 = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [75]:
# Pick a manual seed for randomization
SEED = 645
torch.manual_seed(SEED)
# Create and instance of the model
model = Model()

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)

In [78]:
my_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [79]:
# Change last column from string to integer
my_df['species'] = my_df['species'].replace({'setosa':0, 'versicolor':1, 'virginica':2})

<ipython-input-79-2531912312>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['species'] = my_df['species'].replace({'setosa':0, 'versicolor':1, 'virginica':2})


In [80]:
my_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [81]:
# Train test split. Set X, y
X = my_df.drop('species', axis=1)
y = my_df['species']

In [82]:
# Convert X and Y to numpy arrays.
X = X.values
y = y.values

In [83]:
from sklearn.model_selection import train_test_split

In [84]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [85]:
# Convert to tensors

# X will float tensorrs
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# Y will be long tensors (since Y only has ints, 0 1 2)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [86]:
# Set the criterion of model to measure the error, how far off the predictions are from
criterion = nn.CrossEntropyLoss()

# Choose Optimizer, (Adam Optimizer), lr = learning rate (if error doesnt go down after a bunch of iterations (epocs), lower learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)